In [34]:
#Homework 5
#Tianyang Chen
#An Experiment on Real World Data
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import copy

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [35]:
# import real world data
data = np.loadtxt('blood_train.csv', delimiter=',')
row,col=data.shape
data_x = data[:,:col-1]
data_y = data[:,col-1]

# split data into 5 folds
kf = KFold(n_splits=5, random_state=50, shuffle=True)

# set the probability confidence threshold
threshold=0.9

# start the 5-folf cross validation
error_CV=[]
error_CV_without_SSL=[]
for train_index, test_index in kf.split(data_x):
    
    #split data into training set and testing set
    x_train_CV, x_test_CV = data_x[train_index], data_x[test_index]
    y_train_CV, y_test_CV = data_y[train_index], data_y[test_index]
    
    #for the trainging set, split into labelled set and unlabelled set 
    x_train, x_test, y_train, y_test = train_test_split(x_train_CV, y_train_CV, test_size=0.85, random_state=42)

    #implement Naive Bayes calssifier
    clf = GaussianNB()
    clf.fit(x_train, y_train)
    clf2 = GaussianNB()
    clf2.fit(x_train_CV, y_train_CV)
    error_without_SSL = 1 - clf2.score(x_test_CV, y_test_CV)
    error_CV_without_SSL.append(error_without_SSL)
    
    x_test_copy = copy.deepcopy(x_test)
    y_test_copy = copy.deepcopy(y_test)

    # the self-training rounds
    while len(x_test_copy)>0:  
        clf.fit(x_train, y_train)
        index=[]
        cnt=0
        for i in range(len(x_test_copy)):
            if max(max(clf.predict_proba([x_test_copy[i]]))) > threshold:
                cnt+=1
                # add the data with high probability confidence to training data set
                x_train = np.append(x_train,[x_test_copy[i]], axis=0)
                y_train = np.append(y_train,clf.predict([x_test_copy[i]]), axis=0)
                index.append(i)
        x_test_copy = np.delete(x_test_copy, index, axis=0)
        y_test_copy = np.delete(y_test_copy, index, axis=0)
        clf.fit(x_train, y_train)
        # in the self-traing round, if no data could be added to training data set, stop training
        if cnt==0 or len(x_test_copy)==0:
            
            #compute error on cross validation testing set 
            error_4 = 1 - clf.score(x_test_CV, y_test_CV)
            error_CV.append(error_4)
            print "self-training is completed"
            break

# compute the 5-fold cross validation error
print "Cross Validation error: "+str(np.mean(error_CV))
print "Cross Validation error(without SSL): "+str(np.mean(error_CV_without_SSL))



self-training is completed
self-training is completed
self-training is completed
self-training is completed
self-training is completed
Cross Validation error: 0.255868347339
Cross Validation error(without SSL): 0.247535014006
